In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:

modelname ="GB_MyModel_4ٍEsf__seed_pred 12_1_ep 17_seq 96"
model_weights_path = '/content/drive/MyDrive/Data_FFT_Model/MyModel_4Esfand1403/'+ modelname +'.pth'

model_weights_path_1 = '/content/drive/MyDrive/Data_FFT_Model/MyModel_4Esfand1403_Result/'

Result_name = modelname +'.txt'
R_name_fig_price="price "+modelname+".png"
R_name_fig_load="load "+ modelname + ".png"

pred_file = "pred_"+modelname+".pt"
true_file= "True_"+modelname+".pt"

#imports

In [ ]:
!pip install tqdm

In [5]:
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler,RobustScaler
import pandas as pd

import matplotlib.pyplot as plt

import tqdm
from tqdm import tqdm



#Config

In [6]:

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [7]:
config = dotdict()

config.input_size = 6 #(month, day,hour , ...)
config.output_size = 3

config.hidden_size = 256 #256
config.num_layers =2 #Agar 3 koni bayad be concat kardan akharesh ezafe koni

config.seq_len = 96
config.pred_len = 12

config.learning_rate = 0.001

config.batch_size = 32
config.epochs = 200

config.early_stopping = 7

# Data
### read, preprocess, split, scale, Create Daatasets

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
#----- for data_Load_Forecast

def NEW_for_forecastRead_Data_cleaning():

  data = pd.read_csv('/content/drive/MyDrive/Data_FFT_Model/GB_GBN_withoutTemp_Wind.csv')

  data['date'] = pd.to_datetime(data.date)
  data['year'] = data['date'].dt.year
  data['month'] = data['date'].dt.month
  data['day'] = data['date'].dt.day
  data['hour'] = data['date'].dt.hour

  data = data.drop('date', axis=1)
  data = data.drop('year', axis=1)

  threshold = 2
  #Remove Noise and outliers
  data =data[(data['Price']>0)] 

  print('---- data---- ', data.shape)


  data = data.dropna(axis=0, how='any')

  return data

In [11]:
def Data_forecast_Split_Scales(data):
  #split
  train_size = int( len(data) * 0.8 )
  test_size = len(data) - train_size
  train, test = data[:train_size], data[train_size:]

  print(test.shape)
  scaler_test = StandardScaler()
  test_sc = test.values


  scaler = StandardScaler()#MinMaxScaler()#RobustScaler#ُ

  return train, test , scaler_test


In [ ]:
# dataF , data_Load_Forecast = NEW_for_forecastRead_Data_cleaning()
dataF = NEW_for_forecastRead_Data_cleaning()
print('data_f = ' , dataF.shape)

In [ ]:
train_data_F, test_dataF, scaler = Data_forecast_Split_Scales(dataF)


In [ ]:
print(test_dataF)

In [15]:
def Data_Split_Scales(data):
  #split
  train_size = int( len(data) * 0.8 )
  test_size = len(data) - train_size
  train, test = data[:train_size], data[train_size:]

  print(test.shape)
  scaler_test = StandardScaler()
  test_sc = test.values
  scaler_test.fit(test_sc[:,0:config['output_size']])

  scaler = StandardScaler()#MinMaxScaler()#RobustScaler#ُ

  train = scaler.fit_transform(train.values)
  test = scaler.fit_transform(test.values)

  return train, test , scaler_test


In [ ]:
data  = NEW_for_forecastRead_Data_cleaning()

In [ ]:
train_data_, test_data, scaler = Data_Split_Scales(data)

In [19]:
def MyCreate_dataset(data, seq_len, pred_len, pred_size, step):
  x, y , forecast=[], [], []

  for i in range(0, len(data) - seq_len - pred_len , step):
    feature = data[i: i+seq_len]
    target = data[i+seq_len: i+seq_len+pred_len]

    target = target[:, 0:pred_size]


    x.append(feature)
    y.append(target)

  return torch.tensor(x), torch.tensor(y)

#Metrics

In [20]:
#Metrics
import numpy as np

def RSE(pred, true):
    return np.sqrt(np.sum((true - pred) ** 2)) / np.sqrt(np.sum((true - true.mean()) ** 2))

def CORR(pred, true):
    true = true.reshape(-1, 1)
    pred = pred.reshape(-1, 1)
    u = ((true - true.mean(0)) * (pred - pred.mean(0))).sum(0)
    d = np.sqrt(((true - true.mean(0)) ** 2).sum(0) * ((pred - pred.mean(0)) ** 2).sum(0))
    return (u / d).mean()


def MAE(pred, true):
    return np.mean(np.abs(pred - true))


def MSE(pred, true):
    return np.mean((pred - true) ** 2)


def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))


def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))


def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))


def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)
    rmse = RMSE(pred, true)
    mape = MAPE(pred, true)
    mspe = MSPE(pred, true)
    corr = CORR(pred, true)
    rse = RSE(pred, true)


    return mae, mse, rmse, mape, mspe, corr, rse

#Encoder_Scaled Attention_Decoder (Seq2Seq Model)

In [21]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc_hidden = nn.Linear(hidden_size * 2 * num_layers, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        output_enc, (hidden, cell) = self.lstm(input)  # [batch, seq_len, 2*hidden_size]
        hidden_1dim = self.fc_hidden(hidden.permute(1, 0, 2).reshape(hidden.shape[1], -1))  # [batch, hidden_size]
        output_enc = self.dropout(output_enc)
        return output_enc, hidden, cell, hidden_1dim

In [22]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # Attention mechanism
        self.query = nn.Linear(2 * hidden_size, hidden_size)
        self.key = nn.Linear(2 * hidden_size, hidden_size)
        self.value = nn.Linear(2 * hidden_size, 2 * hidden_size)
        self.dropout = nn.Dropout(dropout)

        # LSTM and output layers
        self.lstm = nn.LSTM(2 * hidden_size + output_size, hidden_size, num_layers=num_layers, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(2 * hidden_size, output_size)

    def forward(self, input, encoder_state, hidden_decoder, cell_decoder, hidden_1dim):
        # Prepare attention mechanism
        query = self.query(encoder_state)  # [batch, seq_len, hidden_size]
        key = self.key(encoder_state)      # [batch, seq_len, hidden_size]
        value = self.value(encoder_state)  # [batch, seq_len, 2*hidden_size]

        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.hidden_size ** 0.5)  # [batch, seq_len, seq_len]
        attn_weights = nn.functional.softmax(scores, dim=-1)  # [batch, seq_len, seq_len]
        attn_weights = self.dropout(attn_weights)

        context_vector = torch.bmm(attn_weights, value)  # [batch, seq_len, 2*hidden_size]
        context_vector = context_vector[:, -1, :].unsqueeze(1)  # [batch, 1, 2*hidden_size]

        # Concatenate context vector with input
        dec_input = torch.cat((context_vector, input.unsqueeze(1)), dim=2)  # [batch, 1, 2*hidden_size + output_size]

        # LSTM step
        output_dec, (hidden_decoder, cell_decoder) = self.lstm(dec_input.permute(1, 0, 2), (hidden_decoder, cell_decoder))
        prediction = self.fc(output_dec.squeeze(0))  # [batch, output_size]

        return prediction, hidden_decoder, cell_decoder

In [23]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, device):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers, dropout=0.1).to(device)
        self.decoder = Decoder(hidden_size, output_size, num_layers, dropout=0.1).to(device)
        self.device = device

    def forward(self, source, target=None):
        batch_size = source.shape[0]
        target_len = target.shape[1] if target is not None else config['pred_len']
        outputs = torch.zeros(target_len, batch_size, config['output_size']).to(self.device)

        # Encoder pass
        out_encoder, hidden, cell, hidden_1dim = self.encoder(source.float())

        # Initialize decoder input
        dec_input = torch.zeros(batch_size, config['output_size']).to(self.device)  # Start token or zeros

        for t in range(target_len):
            teacher_forcing = torch.rand(1) < 0.5 if target is not None else False

            if teacher_forcing and target is not None:
                dec_input = target[:, t, :]  # Use ground truth as next input
            else:
                dec_input = outputs[t - 1] if t > 0 else dec_input  # Use previous prediction

            # Decoder pass
            prediction, hidden, cell = self.decoder(
                dec_input, out_encoder, hidden, cell, hidden_1dim.unsqueeze(0)
            )
            outputs[t] = prediction

        return outputs.permute(1, 0, 2)  # [batch, pred_len, output_size]

#Create Model

In [24]:
momentum = 0 #0.9
alpha = 0.99
eps = 1e-08

model = Seq2Seq(config['input_size'], config['hidden_size'], config['output_size'],
                config['num_layers'], device).to(device)

criteria = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr = config['learning_rate'])


# Test Phases

In [25]:

x_test, y_test = MyCreate_dataset(test_data,config['seq_len'], config['pred_len'], config['output_size'], step = config['pred_len'])

x_test_fft = torch.fft.fft(torch.tensor(x_test))

x_test_fft = torch.real(x_test_fft)

test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_fft, y_test), shuffle=False , batch_size=1, drop_last= True)


<ipython-input-19-63d6c593a390>:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(x), torch.tensor(y)#, torch.tensor(forecast)
<ipython-input-25-a314eb2ad162>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_fft = torch.fft.fft(torch.tensor(x_test))


In [ ]:
# Weghit upload to GPU
# model.load_state_dict(torch.load(model_weights_path))

# Load model weights with map_location set to 'cpu'
model.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')))

In [ ]:
torch.cuda.memory_allocated(device=device)

In [30]:
def test_new(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0

    pred_torch = torch.empty(0)
    true_torch = torch.empty(0)

    pred_array = np.array([])
    true_array = np.array([])

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(tqdm(test_loader, desc='Test')):
            data, target = data.to(device), target.to(device)
            output = model(data.float())
            output = output.to(device)

            loss = criterion(output.float(), target.float())
            test_loss += loss.item()

            true_ = target.squeeze(0)
            true_ = true_.transpose(0, 1)
            true_torch = torch.cat((true_torch.to(device), true_.to(device)), dim=1)


            pred_ = output.squeeze(0)
            pred_ = pred_.transpose(0, 1)


            pred_torch = torch.cat((pred_torch.to(device), pred_.to(device)), dim=1)

            pred_array = np.concatenate((pred_array, pred_.cpu().numpy().flatten()))
            true_array = np.concatenate((true_array, true_.cpu().numpy().flatten()))


    tr_np = true_torch.to('cpu').numpy()
    true_transposed = np.transpose(tr_np)
    true_result = torch.from_numpy(true_transposed)

    true_result = scaler.inverse_transform(true_result.to('cpu'))


    pr_np = pred_torch.to('cpu').numpy()
    pred_transposed = np.transpose(pr_np)
    pred_result = torch.from_numpy(pred_transposed)

    pred_result = scaler.inverse_transform(pred_result.to('cpu'))

    true_result = np.transpose(true_result)
    true_result = torch.tensor(true_result)
    pred_result = np.transpose(pred_result)
    pred_result = torch.tensor(pred_result)


    torch.save(pred_result, model_weights_path_1 + pred_file)
    torch.save(true_result, model_weights_path_1 + true_file)

    print("pred_result = ", pred_result.shape)

    mae, mse, rmse, mape, mspe , corr, rse = metric(pred_array, true_array)

    print('\n++++++++++++++++ Metrics +++++++++++++++++++++\n')
    print('MAE: {}, MSE: {}, RMSE: {}'.format(np.real(mae), np.real(mse), np.real(rmse)))
    print(' MAPE: {}, MSPE: {}'.format( np.real(mape), np.real(mspe)))
    print(' CORR: {}, RSE: {}'.format( np.real(corr), np.real(rse)))

    return test_loss / len(test_loader), pred_torch, true_torch, mae, mse, rmse, mape, mspe , corr, rse ,true_result, pred_result

In [ ]:
avg_loss, pred_torch, true_torch, mae, mse, rmse, mape, mspe , corr, rse ,true_result, pred_result= test_new(model, test_loader, criteria, device)


In [32]:
import os

folder_path = model_weights_path_1
if not os.path.exists(folder_path):
  os.makedirs(folder_path)


#ae, mse, rmse, mape, mspe
str_mtr ="___________________________\n"
str_mtr += "num_epoch = " + str(config['epochs']) + " , batch_size = "+ str(config['batch_size']) + ",\n seq_len = " + str(config['seq_len']) + " , pred_len = " + str(config['pred_len']) +"\n"
str_mtr += " num_layers_encoder = " + str(config['num_layers']) + " , hidden_size = " + str(config['hidden_size']) + " , learning_rate =" + str(config['learning_rate']) +"\n\n"
str_mtr += 'MAE = ' + str(mae) + '\n' + 'MSE = '+ str(mse) + '\n'+ 'RMSE = ' + str(rmse) + '\n'+ 'MAPE = ' + str(mape) + '\n'+ 'MSPE = ' + str(mspe)+"\n"
str_mtr += 'CORR = ' + str(corr) + '\n' + 'RSE = '+ str(rse)

with open( folder_path+Result_name, 'a') as f:
  f.write(str_mtr)
  f.write('\n*******************\n')
  f.close()


print("running is finished")

running is finished
